# Launch (.launch.xml) files 

## Content

1. [Definition](#Definition)
2. [.py launch file](#.py-launch-file)
    1. [Gazebo](#gazebo)
    2. [Rviz](#Rviz)
    3. [RobotStatePublisher](#robot-state-publisher)
    4. [SpawnEntity](#spawn-entity)
    5. [Ros2 controllers](#ros2-controlers)
    6. [Launch description](#launchdescription)


## Definition
.launch files are a special type of files used in ros2, they are used to load diferent type of nodes, comands and logic.

As convention, launch files have the distinc name depending on the file format, the 2 I have used are:

 - *.launch.xml
 - *.launch.py 

## .py launch file

.py launch files normaly contains a lot of logic, allowing it to create various instances of diferent programs, an example of a py file is found in the examen_bringup pkg, in [test_trayectory_controller_scara.launch.py](../../examen_bringup/launch/test_trayectory_controller_scara.launch.py).

There are some definitions that are needed to creare a launch file that shows the xml of a robot and allows it to interact in a virtual environment:


### Gazebo

If you want to test your robot in a virtual environment, a simulator may be needed, in this case, gazebo is the solution:

To setup a gazebo environment ready for ros2, you need to add the next lines at the launch file

```python

world = os.path.join(get_package_share_directory('package-name'),'worlds','world-name')


gazebo = IncludeLaunchDescription(
                PythonLaunchDescriptionSource([os.path.join(
                    get_package_share_directory('gazebo_ros'), 'launch'), '/gazebo.launch.py']),launch_arguments={'world': world}.items())

```

This line of code creates an *IncludeLaunchDescription* object that will be used in the launch description object at the end of the description file to launch an instance of gazebo ready to work with ros2, in a nutshell, it will run this command in the terminal, running a launch file in the gazebo_ros pkg with the parameters 'world' as a filepath: 

```bash

ros2 launch gazebo_ros gazebo.launch.py 'world:=package-name/worlds/world-name'

```


### Rviz

Rviz is a topic visualizer, usefull for visualizing data from sensors.

THis are the basic python lines to get rviz running

```python

rviz_config_path = os.path.join(get_package_share_path('package-name'),'rviz','omni_velocity_rviz.rviz')

config_arg = DeclareLaunchArgument(name = 'rvizconfig', default_value = rviz_config_path)

rviz2_node = Node(
    package="rviz2",
    executable="rviz2",
    arguments=['-d', rviz_config_path])

```

This line of code creates a *Node* object of the rviz2 executable and adds to it a configuration file as arguments, it also can be created in the terminal with the next line:

```bash
os2 run rviz2 rviz2 -d package-name/rviz/omni_velocity_rviz.rviz
```

### Robot state publisher

The robot state publisher is a node in charge of publishing the urdf of a robot into a topic, allowing independent programs like rviz to create the robot, it is also used by the [SpawnEntity](#spawn-entity) node for gazebo.

```python

urdf_path = os.path.join(get_package_share_path('package-name'),'urdf','omni_velocity_controller.xacro')

robot_description = ParameterValue(Command(['xacro ', urdf_path]), value_type=str)

node_robot_state_publisher = Node(
        package='robot_state_publisher',
        executable='robot_state_publisher',
        output='screen',
        parameters=[{'robot_description':robot_description}]
    )

```

This lines of code extract the urdf from a file given a path, then it calls the xacro function with the urdf as argument and returns it as a string, to finally create a node of the robot_state_publisher file from the robot_state_publisher package with the xacro as a parameter.

It is also posible to use de terminal to run this node:

```bash
ros2 run robot_state_publisher robot_state_publisher src/package-name/urdf/fullxacro.xacro 
```


### Spawn entity

The Spawn_entíty is a node in charge of taking the robot description published in a topic (in this case, by the [robot-state-publisher](#robot-state-publisher)) and show it in gazebo.

```python

spawn_entity = Node(package='gazebo_ros', executable='spawn_entity.py',
                    arguments=['-topic', 'robot_description',
                                '-entity', 'omni'],
                    output='screen')

```

This line of code creates a node of the spawn_entity.py file in the gazebo_ros package, and adds as arguments the topic to get the urdf from, and the entity name  

The code that represents this line of code can be seen below:

```bash
ros2 run gazebo_ros spawn_entity.py -topic robot_description -entity omni_bot
```

### Ros2 controlers

```bash
ros2 run gazebo_ros spawn_entity.py -topic robot_description -entity omni_bot
```

### LaunchDescription